In [1]:
import geopandas as gpd
import rasterio as rio
from rasterio.enums import Resampling

#### Resample

In [2]:
data_path = r'C:\SKOLA\02_DOKTORAT\ml_dpz\lcz\berlin_data\20m_bands_mosaic.tif'
mosaic_10m = r'C:\SKOLA\02_DOKTORAT\ml_dpz\lcz\berlin_data\10m_bands_mosaic.tif'

In [3]:
with rio.open(data_path) as src:
    resampling=src.res
    print(resampling)
    img=src.read()
    print(img.shape)
    
with rio.open(mosaic_10m) as mos_10m:
    wanted=mos_10m.res
    print(wanted)
    img=mos_10m.read()
    print(img.shape)

(19.983280866784742, 19.983280866784742)
(11, 3334, 3218)
(9.991639925114471, 9.991639925114471)
(6, 6667, 6435)


In [4]:
#upscale_factor = 2
upscale_factor_height=resampling[0]/wanted[0]
upscale_factor_width=resampling[1]/wanted[1]

with rio.open(data_path) as src:
    meta = src.meta
    
    # resample data to target shape
    resampled = src.read(
        out_shape=(
            src.count,
            int(src.height * upscale_factor_height),
            int(src.width * upscale_factor_width)
        ),
        resampling=Resampling.bilinear
    )

    print('Shape before resample:', src.shape)
    print('Shape after resample:', resampled.shape[1:])

    # scale image transform
    transform = src.transform * src.transform.scale(
        (src.width / resampled.shape[-1]),
        (src.height / resampled.shape[-2])
    )

    print('Transform before resample:\n', src.transform, '\n')
    print('Transform after resample:\n', transform)


Shape before resample: (3334, 3218)
Shape after resample: (6668, 6436)
Transform before resample:
 | 19.98, 0.00, 360268.74|
| 0.00,-19.98, 5855935.51|
| 0.00, 0.00, 1.00| 

Transform after resample:
 | 9.99, 0.00, 360268.74|
| 0.00,-9.99, 5855935.51|
| 0.00, 0.00, 1.00|


In [5]:
out_meta_resampled = meta.copy()
out_meta_resampled.update({
    'height': resampled.shape[1],
    'width': resampled.shape[2],
    'transform': transform
})

In [6]:
with rio.open('20m_mosaic_resampled_to_10m.tif', 'w', **out_meta_resampled) as dst:
    dst.write(resampled)

In [8]:
with rio.open('20m_mosaic_resampled_to_10m.tif') as resam:
    print(resam.res)

(9.991640433392371, 9.991640433392371)


In [ ]:
9.991639925114471, 9.991639925114471